## *Dropout*
En aquest exercici, afegirem l'abandonament al model de [Spotify](../data/spotify.csv) i veurem com la normalització per lots ens pot permetre entrenar models amb èxit en conjunts de dades difícils.

### Preparem les dades

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

spotify = pd.read_csv('data/spotify.csv')

X = spotify.copy().dropna()
y = X.pop('track_popularity')
artists = X['track_artist']

features_num = ['danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms']
features_cat = ['playlist_genre']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

# We'll do a "grouped" split to keep all of an artist's songs in one
# split or the other. This is to help prevent signal leakage.
def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, artists)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100 # popularity is on a scale 0-100, so this rescales to 0-1.
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

### Pregunta 1
Ací tenim un model de l'anterior activitat. Executa'l i calcula el seu `val_loss` mínim. Representa gràficament les corbes d'aprenentatge.
```python
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
```

### Pregunta 2
Afegiu dues capes d'abandonament, una després de la capa Densa amb 128 unitats i una després de la capa Densa amb 64 unitats. Estableix la taxa d'abandonament en les dues capes a 0,3. Representa de nou les corbes d'aprenentatge i calcula la mínima `val_loss`.

### Pregunta 3
Representa en dos subplots les corbes d'aprenentatge de les preguntes anteriors. Comenta les gràfiques.